In [1]:
import pandas as pd
import numpy as np
#from google.colab import drive
from statsmodels.tools.eval_measures import rmse

In [2]:
#drive.mount('/content/drive')

In [3]:
def df_split(df):
  categoria = []
  for x in df['category_name']:
    categoria.append(x.split('/'))
  return categoria


def sub_categoria(df):  
  lista = df_split(df)
  for y in range(len(lista)):
    lista[y] = lista[y][1]
  return lista

In [4]:
filtro_sport_outdoor = pd.read_csv('https://github.com/AugustoCRX/bootcamp_blue/blob/main/novos_arquivos_treino/Limpeza%20v3/filtro_Sports%20_%20Outdoors_train_v3.csv?raw=true')

In [5]:
filtro_sport_outdoor.head()

,name,item_condition_id,category_name,brand_name,price,shipping,item_description,date,stock,date2,day,month,year,description_clear,name_clear,brands_filter
0,Girls cheer and tumbling bundle of 7,3,Sports & Outdoors/Apparel/Girls,Soffe,6.0,1,You get three pairs of Sophie cheer shorts siz...,25-7-2018,3,2018-07-25,25,7,2018,get three pair sophi cheer short size small me...,girl cheer tumbl bundl 7,basics_brand
1,Girls Nike Pro shorts,3,Sports & Outdoors/Apparel/Girls,Nike,19.0,0,Girls Size small Plus green. Three shorts total.,12-4-2018,1,2018-12-04,4,12,2018,girl size small plu green three short total,girl nike pro short,nike
2,Jillian Michaels bodyshred Fitness DVD,1,Sports & Outdoors/Exercise/Fitness technology,Other,39.0,0,This boxed set of 12 DVD’s includes the Fitnes...,8-6-2018,1,2018-08-06,6,8,2018,box set 12 dvd includ fit guid meal plan rotat...,jillian michael bodyshr fit dvd,upper_pop_brand
3,Crimson Tide Alabama T-shirt,3,Sports & Outdoors/Fan Shop/NCAA,Other,10.0,0,"Size M (8), cuffs show wear and letters are no...",14-6-2018,7,2018-06-14,14,6,2018,size 8 cuff show wear letter peel came like,crimson tide alabama t-shirt,popular_brand
4,Marucci Pro Cut,3,Sports & Outdoors/Team Sports/Baseball & Softball,Marucci,46.0,0,33/30,4-8-2018,16,2018-04-08,8,4,2018,3330,marucci pro cut,upper_pop_brand


In [6]:
filtro_sport_outdoor['sub_category_name'] = sub_categoria(filtro_sport_outdoor)

In [7]:
filtro_sport_outdoor.sub_category_name.value_counts()

Exercise       5772
Fan Shop       4479
Outdoors       3301
Apparel        2210
Team Sports    1699
Golf            647
Footwear        420
Other           289
Name: sub_category_name, dtype: int64

# Modelo 01
Regressão Linear sem restrição de preço e com variaveis dummies nas variaveis categoricas sub_category_name, shipping e brands_filter

In [8]:
import statsmodels.api as sm

In [9]:
filtro_sport_outdoor = pd.get_dummies(filtro_sport_outdoor, columns=['sub_category_name','brands_filter','shipping'])

In [10]:
X = filtro_sport_outdoor.drop(['name','category_name','item_description','date','stock','date2','day','month','year','stock','price','brand_name','description_clear','name_clear'],axis=1)
y = filtro_sport_outdoor.price
X = sm.add_constant(X)
m1_rl = sm.OLS(exog=X,endog=y)
m1_result = m1_rl.fit()
print(m1_result.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.515
Model:                            OLS   Adj. R-squared:                  0.514
Method:                 Least Squares   F-statistic:                     643.7
Date:                Thu, 13 Oct 2022   Prob (F-statistic):               0.00
Time:                        17:21:48   Log-Likelihood:                -83919.
No. Observations:               18817   AIC:                         1.679e+05
Df Residuals:                   18785   BIC:                         1.682e+05
Df Model:                          31                                         
Covariance Type:            nonrobust                                         
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
const   

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [11]:
filtro_sport_outdoor.price.describe()

count    18817.000000
mean        26.310677
std         30.046665
min          3.000000
25%         11.000000
50%         17.000000
75%         29.000000
max        915.000000
Name: price, dtype: float64

In [12]:
m1_ypred = m1_result.predict(X)
m1_RMSE = rmse(y,m1_ypred)
m1_RMSE.round(2)

20.92

# Modelo 02

Regressão Linear com restrição de preço ate 200 dolares e com variaveis dummies nas variaveis categoricas sub_category_name, shipping e brands_filter

In [13]:
filtro_sport_outdoor_ajustado = filtro_sport_outdoor[filtro_sport_outdoor['price'] <= 200]

In [14]:
X1 = filtro_sport_outdoor_ajustado.drop(['name','category_name','item_description','date','stock','date2','day','month','year','stock','price','brand_name','description_clear','name_clear'],axis=1)
y1 = filtro_sport_outdoor_ajustado.price
X1 = sm.add_constant(X1)
m2_rl = sm.OLS(exog=X1,endog=y1)
m2_result = m2_rl.fit()
print(m2_result.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.548
Model:                            OLS   Adj. R-squared:                  0.548
Method:                 Least Squares   F-statistic:                     757.9
Date:                Thu, 13 Oct 2022   Prob (F-statistic):               0.00
Time:                        17:21:48   Log-Likelihood:                -79808.
No. Observations:               18761   AIC:                         1.597e+05
Df Residuals:                   18730   BIC:                         1.599e+05
Df Model:                          30                                         
Covariance Type:            nonrobust                                         
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
const   

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [15]:
filtro_sport_outdoor_ajustado['price'].describe()

count    18761.000000
mean        25.495496
std         25.340260
min          3.000000
25%         11.000000
50%         17.000000
75%         29.000000
max        200.000000
Name: price, dtype: float64

In [16]:
m2_ypred = m2_result.predict(X1)
m2_RMSE = rmse(y1,m2_ypred)
m2_RMSE.round(2)

17.03

# Modelo 03

Arvore de Regressão usando como criterio de seleção dos nós a métrica R²

In [17]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error,mean_squared_error

In [18]:
X2 = filtro_sport_outdoor.drop(['name','category_name','item_description','date','stock','date2','day','month','year','stock','price','brand_name','description_clear','name_clear'],axis=1)
y2 = filtro_sport_outdoor.price
m3_dtr = DecisionTreeRegressor(random_state=10)
result2 = m3_dtr.fit(X2,y2)
m3_r2 = result2.score(X2,y2)
m3_ypred = result2.predict(X2)
m3_mse = mean_squared_error(y2,m3_ypred)
m3_rmse = m3_mse**0.5
print(f"R² = {m3_r2.round(2)}")
print(f"MSE = {m3_mse.round(2)}")
print(f"RMSE = {m3_rmse.round(2)}")

R² = 0.56
MSE = 395.28
RMSE = 19.88


# Modelo 04

Arvore de Regressão com restrição de preço ate 200 dolares e com variaveis dummies nas variaveis categoricas sub_category_name, shipping e brands_filter

In [19]:
X3 = filtro_sport_outdoor_ajustado.drop(['name','category_name','item_description','date','stock','date2','day','month','year','stock','price','brand_name','description_clear','name_clear'],axis=1)
y3 = filtro_sport_outdoor_ajustado.price
m4_dtr = DecisionTreeRegressor(random_state=10)
result3 = m4_dtr.fit(X3,y3)
m4_r2 = result3.score(X3,y3)
m4_ypred = result3.predict(X3)
m4_mse = mean_squared_error(y3,m4_ypred)
m4_rmse = m4_mse**0.5
print(f"R² = {m4_r2.round(2)}")
print(f"MSE = {m4_mse.round(2)}")
print(f"RMSE = {m4_rmse.round(2)}")

R² = 0.59
MSE = 260.21
RMSE = 16.13


# Resultados

| Modelo | R² |RMSE | Observação |
| --- | --- | --- | --- |
| Modelo 01 | 51,5% | 20,92 | Regressão Linear |
| Modelo 02 | 54,8% | 17,03 | Regressão Linear (preço <= 200) |
| Modelo 03 | 56,0% | 19,88 | Arvore Regressão (criterio = R²) |
| Modelo 04 | 59,0% | 16,13 | Arvore Regressão (criterio = R², preço <= 200) |